In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
import itertools
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=(224,224,3)))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 64)       

In [6]:
transfer_model = model

In [7]:
import os
dirs = os.listdir(".\\train")
for each in dirs:
    print(each+ " " +str(len(os.listdir(".\\train\\" + each))))

BCC 2624
BKL 2624
MEL 2624


In [8]:
import numpy as np
import time
import uuid
import cv2
def preprocess_img(img):
    img=img.astype(np.uint8)
    equ=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    clahe = cv2.createCLAHE(clipLimit=3., tileGridSize=(8,8))
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l2 = clahe.apply(l)
    lab = cv2.merge((l2,a,b))
    img2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return equ.astype(np.float32)
def test():
    print("hi\n")
from keras.callbacks import ModelCheckpoint
filepath="EF2-{epoch:02d}-auc_{val_auc:2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.compile(optimizer="adam",loss = "categorical_crossentropy",metrics=["accuracy",tf.keras.metrics.AUC(multi_label = True)])
classifier=transfer_model
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        preprocessing_function=preprocess_img,
        horizontal_flip=True,
#         preprocessing_function=preprocess_img,
        validation_split=0.2)
total_images = 25331

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_img)

training_set= train_datagen.flow_from_directory(
        './train',
        subset="training",
        target_size=(224, 224),
        batch_size=32)
validatin_set = train_datagen.flow_from_directory(
        "./train",
        subset = "validation",
        target_size=(224, 224),
        batch_size=32)


# # Class weights
# from sklearn.model_selection import train_test_split
# from sklearn.utils import class_weight

# from collections import Counter
# class_weights = class_weight.compute_class_weight('balanced',
#                                                   classes=np.unique(training_set.classes), 
#                                                   y=training_set.classes) 

# unique_class_weights = np.unique(training_set.classes)
# class_weights_dict   = { unique_class_weights[i]: w for i,w in enumerate(class_weights) }

# print('\nCLASS WEIGHTS: {}\n'.format(class_weights))
# print(np.unique(training_set.classes))
# print(training_set.classes)
# print(unique_class_weights)
# print(Counter(training_set.classes).keys())   # equals to list(set(x))
# print(Counter(training_set.classes).values()) # counts the elements' frequency

Found 6300 images belonging to 3 classes.
Found 1572 images belonging to 3 classes.


In [9]:
hist=classifier.fit(
        training_set,
        steps_per_epoch=6300//32,
        epochs=20,
        validation_data=validatin_set,
        validation_steps=1572//32,callbacks=callbacks_list)
print(hist.history)

Epoch 1/20


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[32,112,112,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_1/dropout_3/dropout/random_uniform/RandomUniform (defined at \AppData\Local\Temp/ipykernel_21036/3519108300.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[assert_less_equal/Assert/AssertGuard/pivot_f/_13/_63]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[32,112,112,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_1/dropout_3/dropout/random_uniform/RandomUniform (defined at \AppData\Local\Temp/ipykernel_21036/

In [1]:
classifier.save("trained_effb_0.hdf5")

NameError: name 'classifier' is not defined

In [ ]:
from keras.models import load_model
saved = load_model("eficientnetb0-09-acc_0.43.hdf5")
mdl = Model(inputs=saved.input,outputs=saved.output)
from keras.optimizers import Adam
mdl.compile(optimizer="adam",loss = "categorical_crossentropy",metrics=["accuracy"])
mdl.fit_generator(
        training_set,
        steps_per_epoch=8377//16,
        epochs=10,
        validation_data=validatin_set,
        validation_steps=2092//16,callbacks=callbacks_list,class_weight=class_weights_dict)